In [1]:
import os
import warnings
import numpy as np
import pandas as pd
warnings.simplefilter("ignore")
warnings.filterwarnings("ignore")
warnings.filterwarnings('ignore',category=FutureWarning)
from typing import Tuple, List, NoReturn, Union, Any

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.losses import SparseCategoricalCrossentropy

## Load data

In [4]:
def load_parsed(dump_features_location: str = "../train_data",
                dump_features_fn: str = "train_dataset.npz"):
    data = np.load(os.path.join(dump_features_location, dump_features_fn))
    features, outputs = data['features'], data['outputs']
    print(f"Features shape: {features.shape}")
    print(f"Outputs shape: {outputs.shape}")

    return features, outputs

In [5]:
data_input, data_output = load_parsed("train_data", "train_dataset.npz")

Features shape: (274818, 28, 11)
Outputs shape: (274818, 28)


## Create model

In [11]:
class CNN_Net(object):
    FIRST_LAYER_FILTERS = 16
    FIRST_LAYER_KERNEL = 2
    SECOND_LAYER_FILTERS = 32
    SECOND_LAYER_KERNEL = 2

    def __init__(self, input_size: Tuple[int, int], output_size: int,
                 cached_model: bool, cached_model_path: str, seed=None):
        np.random.seed(seed)
        if cached_model:
            # returns an already compiled model
            self._model = load_model(cached_model_path)
        else:
            # returns an already compiled model
            self._model = self.create_model(input_size, output_size,
                                            loss=SparseCategoricalCrossentropy(),
                                            optimizer="adam",
                                            metrics=['accuracy'], verbose=True)


    def create_model(self, input_size: Tuple[int, int], output_size: int,
                     loss: Union[str, Any], optimizer: str, metrics: List[str],
                    verbose: bool):

        model = Sequential()
        model.add(Conv1D(self.FIRST_LAYER_FILTERS, self.FIRST_LAYER_KERNEL,
                               activation='relu', batch_input_shape=(None, *input_size))) #input_shape=input_size
        model.add(BatchNormalization())
        model.add(MaxPooling1D())
        model.add(Conv1D(self.SECOND_LAYER_FILTERS, self.SECOND_LAYER_KERNEL,
                               activation='relu'))
        model.add(BatchNormalization())
        model.add(GlobalAveragePooling1D())
        model.add(Dropout(0.5))
        model.add(Dense(output_size, activation='sigmoid'))
        if verbose:
            print("Model created:")
            print(model.summary())

        self._loss = loss
        self._metrics = metrics
        model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
        return model

    def train(self, input_train, target_train, validation_split: float,
              n_epochs: int, batch_size: int, verbose: int,
              model_version: str)-> NoReturn:

        # Fit data to model
        history = self._model.fit(input_train, target_train,
                                  batch_size=batch_size,
                                  epochs=n_epochs,
                                  verbose=verbose,
                                  validation_split=validation_split,
                                  shuffle=True)

        # Plot history
        print(f"History consist of: {history.keys()}")
        current_directory = os.path.dirname(os.path.abspath(__file__))
        hist_df = pd.DataFrame(history, columns=self._metrics + ['loss', 'val_loss'])
        fig = hist_df.plot(x=hist_df.index, y=['loss', 'val_loss']).get_figure()
        curve_path = os.path.join(current_directory, os.path.pardir, "models", "training_plot.png")
        fig.savefig(curve_path)

        # Save model
        self.save(model_save_fn="cnn_model_" + model_version + ".hd5")


    def predict(self, input_data):
        predictions = self._model.predict(input_data)
        return predictions


    def save(self, model_save_path="../models",
             model_save_fn="cnn_model.hd5"):
        self._model.save(os.path.join(model_save_path, model_save_fn))

In [15]:
CNN_Net(input_size=data_input.shape[1:], output_size=28, cached_model=False, cached_model_path="")

Model created:
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 27, 16)            368       
_________________________________________________________________
batch_normalization_3 (Batch (None, 27, 16)            64        
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 13, 16)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 12, 32)            1056      
_________________________________________________________________
batch_normalization_4 (Batch (None, 12, 32)            128       
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout (Dropout)            (None, 32)

In [14]:
data_input.shape[1:]

(28, 11)